In [1]:
import numpy as np
import pandas as pd
import sklearn
import torch

In [17]:
%config Completer.use_jedi = False

In [3]:
torch.manual_seed(1)

In [4]:
x_train = np.array(pd.read_csv("./2021-ai-w6-p1/pca_train.csv"))
y_train = np.array(pd.read_csv("./2021-ai-w6-p1/train_label.csv")['label'])
x_test = np.array(pd.read_csv("./2021-ai-w6-p1/pca_test.csv"))
submit_df = pd.read_csv("./2021-ai-w6-p1/sample_submission.csv")

In [5]:
# preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

x_train = torch.Tensor(x_train).cuda()
x_test = torch.Tensor(x_test).cuda()

le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
y_train = torch.LongTensor(y_train).cuda()
print(x_train.shape, y_train.shape)

torch.Size([27671, 256]) torch.Size([27671])


In [6]:
layer1 = torch.nn.Linear(256,128,bias=True).cuda()
layer2 = torch.nn.Linear(128,128,bias=True).cuda()
layer3 = torch.nn.Linear(128,128,bias=True).cuda()
layer4 = torch.nn.Linear(128,128,bias=True).cuda()
layer5 = torch.nn.Linear(128,256,bias=True).cuda()
layer6 = torch.nn.Linear(256,256,bias=True).cuda()
layer7 = torch.nn.Linear(256,len(le.classes_),bias=True).cuda()

model = torch.nn.Sequential(layer1,layer2,layer3,layer4,layer5,layer6,layer7).cuda()

In [7]:
optim = torch.optim.Adam(model.parameters(),lr=1e-4)
loss = torch.nn.CrossEntropyLoss()
epochs =1000

In [8]:
for epoch in range(epochs+1):
    optim.zero_grad()
    
    h = model(x_train)
    cost = loss(h,y_train)
    
    cost.backward()
    optim.step()
    if epoch % (epochs / 10) == 0:
        print("epochs =", str(epochs), "| epoch =", str(epoch), "| cost =", cost)

epochs = 1000 | epoch = 0 | cost = tensor(5.2895, device='cuda:0', grad_fn=<NllLossBackward>)
epochs = 1000 | epoch = 100 | cost = tensor(3.7802, device='cuda:0', grad_fn=<NllLossBackward>)
epochs = 1000 | epoch = 200 | cost = tensor(1.9737, device='cuda:0', grad_fn=<NllLossBackward>)
epochs = 1000 | epoch = 300 | cost = tensor(1.6456, device='cuda:0', grad_fn=<NllLossBackward>)
epochs = 1000 | epoch = 400 | cost = tensor(1.5144, device='cuda:0', grad_fn=<NllLossBackward>)
epochs = 1000 | epoch = 500 | cost = tensor(1.4376, device='cuda:0', grad_fn=<NllLossBackward>)
epochs = 1000 | epoch = 600 | cost = tensor(1.3759, device='cuda:0', grad_fn=<NllLossBackward>)
epochs = 1000 | epoch = 700 | cost = tensor(1.3233, device='cuda:0', grad_fn=<NllLossBackward>)
epochs = 1000 | epoch = 800 | cost = tensor(1.2790, device='cuda:0', grad_fn=<NllLossBackward>)
epochs = 1000 | epoch = 900 | cost = tensor(1.2407, device='cuda:0', grad_fn=<NllLossBackward>)
epochs = 1000 | epoch = 1000 | cost = tens

In [15]:
with torch.no_grad():
    predict = model(x_test)
    predict = torch.argmax(predict, dim =1)
    predict = le.inverse_transform(predict.cpu().detach())
submit_df['label'] = predict

In [16]:
submit_df

,id,label
0,0,113
1,1,83
2,2,68
3,3,28
4,4,121
...,...,...
13624,13624,77
13625,13625,69
13626,13626,23
13627,13627,68
